In [18]:
!pip install -q openai

In [52]:
from openai import OpenAI
import requests
import json

client = OpenAI(
    base_url = "https://openrouter.ai/api/v1",
    api_key=""
)
API_KEY = ""
API_HOST = "https://api.stablecog.com"
API_ENDPOINT = "/v1/image/generation/create"
API_URL = f"{API_HOST}{API_ENDPOINT}"

In [53]:
def call_agent(user_input):
  print("Writing prompt...")
  completion = client.chat.completions.create(
    model = 'openai/gpt-4o-mini',
    messages = [
        {"role":"system","content":"You are an expert prompt writer. You specialize in writing descriptive prompts for the AI image tool stable diffusion."},
        {"role":"user","content":f"Write a concise prompt for this '{user_input}'. Only give the prompt as output, no extra information."}
    ]
  )
  prompt = completion.choices[0].message.content
  print(f"Prompt written: {prompt}")
  return prompt

In [54]:
def generate_image(prompt):
    """Generate an image using Stablecog API with a custom prompt."""
    req = {
        "prompt": prompt,
        "num_outputs": 1
    }

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    res = requests.post(API_URL, data=json.dumps(req), headers=headers)

    try:
        res_json = res.json()
        # print(json.dumps(res_json, indent=2))  # Removed for concise output
        if res_json and "outputs" in res_json and len(res_json["outputs"]) > 0 and "image_url" in res_json["outputs"][0]:
            print(res_json["outputs"][0]["image_url"])
            return None
        else:
            print("Image URL not found in the response.")
            return None
    except json.JSONDecodeError:
        print("Error decoding JSON response:", res.text)
        return None

def get_text():
  text = input("Enter your image text: ")
  return text

In [22]:
user_input = get_text()
prompt = call_agent(user_input)
generate_image(prompt)

Enter your image text: 
Writing prompt...


AuthenticationError: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}

In [55]:
def decision_agent(user_text):
  completion = client.chat.completions.create(
    model = 'openai/gpt-4o-mini',
    messages = [
        {"role":"system","content":""""You are a decision-making agent. Analyze the user's input and decide whether to use the "generate_image" tool or the "web_browsing" tool.
                                   Output your decision in JSON format with 'reasoning' and 'tool' fields.
                                   Example format:
                                   {
                                    "reasoning": "Thorough explanation for the decision...",
                                    "tool": "generate_image" or "web_browsing"
                                   }
                                   Do NOT output any other text other than the JSON."""},
        {"role":"user","content":f"Decide the best tool for this task: {user_text}. Output ONLY the JSON with 'reasoning' and 'tool' fields."}
    ]
  )
  return completion.choices[0].message.content

In [56]:
result = decision_agent("I would like to visualize how dubai covered in snow night look like")
print(result)

{
  "reasoning": "The user is asking for a visualization of Dubai covered in snow at night, which is a creative task that can be accomplished with image generation to provide a unique representation of that scenario.",
  "tool": "generate_image"
}


In [64]:
def web_browsing(query):
  print("Browsing the web...")
  response = client.chat.completions.create(
    model = 'openai/gpt-oss-20b:free',
    messages = [
        {"role":"system","content":"You are an AI assistant providing information based on web searches"},
        {"role":"user","content":query}
    ]
  )
  print(response.choices[0].message.content)

In [58]:
def parse_execute(result, user_input):
  data = json.loads(result)
  tool = data.get('tool')
  if tool=="generate_image":
    print("Generating image...")
    prompt = call_agent(user_input)
    generate_image(prompt)
  elif tool=="web_browsing":
    web_browsing(user_input)
  else:
    return f"Error: Unknown tool '{tool}'"

In [65]:
print("Agent: How can i help you?")
user_ip = input("User: ")
result = decision_agent(user_ip)
parse_execute(result, user_ip)

Agent: How can i help you?
User: I am wondering what might happen about future interviews
Browsing the web...
Sure! “Future interviews” can mean a lot of things—whether you’re talking about job‑interview trends, media or public‑figure Q&A, or even new tools for coaching you through that process. Below, I’ve broken it into a few practical angles so you can see what’s likely on the horizon and how you might prepare for it.

---

## 1. **Job Interview Trends**

| Trend | What It Looks Like | Why It Matters | Practical Tips |
|-------|-------------------|----------------|----------------|
| **Hybrid-Bare‑Bones Video** | Short, live‑stream interviews that blur “live” and “pre‑recorded.” Companies clip authentic moments, sometimes with audience reactions. | Gives recruiters a “real‑time” feel while limiting time on a talk‑show‑style. | *Own your space*: quiet, neutral background, good audio, eye‑contact via camera. |
| **AI‑Scrutinized CVs** | Natural‑language‑processing tools scan resumes f